In [2]:
Input_WHs = [
    [640, 320],  
    [672, 352],   
    [704, 384],  
    [736, 416],   
    [768, 448],   
    [800, 480],
    [832, 512],   
    [864, 544],  
    [896, 576],   
    [928, 608],   
    [960, 640],   
    [992, 672],  
    [1064, 704],  
    [1064, 736],  
    [1064, 608],  
    [1088, 608]]  

In [3]:
from collections import OrderedDict, defaultdict
import glob
import math
import os
import os.path as osp
import random
import copy
import time
import warnings

import cv2
import numpy as np
import torch

In [4]:
class LoadImagesAndLabels:
    def __init__(self, path, img_size = (1088, 608), augment=False, transforms = None):
        with open(path, 'r') as file:
            self.img_files = file.readlines()
            self.img_files = [ele.replace("\n", "") for ele in self.img_files]
            self.img_files = list(filter(lambda x: len(x)>0, self.img_files))
        
        self.label_files = [x.replace("images", "labels_with_id").
                           replace(".jpg", ".txt").
                           replace(".png", ".txt")
                           for x in self.img_files]
        self.nF = len(self.img_files)
        
        self.width = img_size[0]
        self.height = img_size[1]
        
        self.augment = augment
        self.transforms = transforms
        
    def __getitem__(self, files_index):
        img_path = self.img_files[files_index]
        label_path = self.label_files[files_index]
        return self.get_data(img_path, label_path)
    
    def get_data(self, img_path, label_path, width=None, height=None):
        if height is None or width is None:
            width = self.width
            height = self.height
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError("file is not available!")
        augment_hsv = True
        if self.augment and augment_hsv:
            fraction = 0.5
            img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            S = img_hsv[:,:,1].astype(np.float32)
            V = img_hsv[:,:,2].astype(np.float32)
            
            a = (random.random()*2-1)*fraction+1
            S*=a
            if a>1:
                np.clip(S, a_min=0, a_max=255, out=S)
            
            a = (random.random()*2-1)*fraction+1
            V*=a
            if a>1:
                np.clip(V, a_min=0, a_max=255, out=V)
            
            img_hsv[:,:,1] = S.astype(np.uint8)
            img_hsv[:,:,2] = V.astype(np.uint8)
            cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR, dst=img)
        
        h, w = img.shape[:2]
        img, ratio, pad_w, pad_h = letterbox(img, height, width, color=(127.5, 127.5, 127.5))
        
        if os.path.isfile(label_path):
            labels_0 = np.loadtxt(label_path, dtype=np.float32).reshape((-1,6))
            
            labels = labels_0.copy()
            labels[:,2] = ratio*w*(labels_0[:,2] - labels_0[:,4]/2) + pad_w
            labels[:,3] = ratio*w*(labels_0[:,3] - labels_0[:,5]/2) + pad_h
            labels[:,4] = ratio*w*(labels_0[:,2] - labels_0[:,4]/2) + pad_w
            labels[:,5] = ratio*w*(labels_0[:,3] - labels_0[:,5]/2) + pad_h
        else:
            labels = np.array([])
        
        if self.augment:
            img, labels, M = random_affine(img, labels, 
                                          degree = (-5,5),
                                          translate = (-0.1, 0.1),
                                          scale = (0.5, 1.2))
        num_labels = len(labels)
        if num_labels>0:
            labels[:, 2:6] = xyxy2xywh(labels[:, 2:6].copy())
            
            labels[:,2] /= width
            labels[:,3] /= height
            labels[:,4] /= width
            labels[:,5] /= height
        
        if self.augment:
            lf_flip = True
            if lf_flip and (random.random()>0.5):
                img = np.fliplr(img)
                if num_labels > 0:
                    labels[:,2] = 1 - labels[:,2]
        
        img = np.ascontiguousarray(img[:,:,::-1])
        if self.transforms:
            img = self.transforms(img)
        return img, labels, img_path, (h,w)
    
    def __len__(self):
        return self.nF  

In [ ]:
class MultiScaleJD(LoadImagesAndLabels):
    mean = None
    std = None
    
    def __init__(self, opt, root, paths, img_size = (1088, 608),
                augment = False, transforms = None):
        self.opt = opt
        self.img_files = OrderedDict()
        self.label_files = OrderedDict()
        self.tid_num = OrderedDict()
        self.tid_start_index =OrderedDict()
        self.num_classes = list(range(10))
        # '0,1,2,3,4,5,6,7,8,9'
        
        if opt.input_wh[0] != img_size[0] or opt.input_wh[1] != img_size[1]:
            opt.input_wh[0], opt.input_wh[0] = img_size[0], img_size[1]
        
        self.default_input_wh = opt.input_wh
        
        self.width, self.height = self.default_input_wh[0], self.default_input_wh[1]
        self.batch_i_to_scale_i = defaultdict(int)
        
        self.paths = paths
        for ds, path in self.paths.items():
            with open(path, 'r') as file:
                self.img_files[ds] = file.readlines()
                self.img_files[ds] = [osp.join(root, x.strip()) for x in self.img_files[ds]]
                self.img_files[ds] = list(filter(lambda x: len(x)>0, self.img_files[ds]))
        
            self.label_files = [x.replace("images", "labels_with_id").
                               replace(".jpg", ".txt").
                               replace(".png", ".txt")
                               for x in self.img_files[ds]]
            
            print("Total {} image files in {} dataset".format(len(self.label_files[ds]), ds))
        
        if opt.id_weight > 0:
            for ds, label_paths in self.label_files.items():
                max_ids_dict = defaultdict(int)
                
                for lp in label_paths:
                    if not os.path.isfile(lp):
                        print("file is not available!")
                        continue
                    lb = np.loadtxt(lp)
                    if len(lb) < 1:
                        continue
                    lb = lb.reshape((-1,6))
                    for item in lb:
                        if item[1] > max_ids_dict[int(item[0])]:
                            max_ids_dict[int(item[0])] = item[1]
                
                self.tid_num[ds] = max_ids_dict 
            
            self.tid_start_idx_of_cls_ids = defaultdict(int)
            last_idx_dict = defaultdict(int)
            
            for k, v in self.tid_num.items():
                for cls_id, id_num in v.items():
                    self.tid_start_idx_of_cls_ids[k][cls_id] = last_idx_dict[cls_id]
                    last_idx_dict[cls_id] += id_num
            
            self.nID_dict = defaultdict(int)
            for k, v in last_idx_dict.items():
                self.nID_dict[k] = int(v)
            
            self.nds = [len(x) for x in self.img_files.values()]
            self.cds = [sum(self.nds[:i]) for i in range(len(self.nds))]
            self.nF = sum(self.nds)
            self.max_objs = opt.K
            self.augment = augment
            self.transforms = transforms
            
            self.input_multi_scales = None
            if opt.gen_scale:
                self.gen_multi_scale_input_whs()
            
            self.rand_scale()

            def rand_scale(self):
        if self.input_multi_scales is None:
            self.num_batches = self.nF // self.batch_size + 1
            for batch_i in range(self.num_batches):
                rand_batch_idx = np.random.randint(0, self.num_batches)
                rand_scale_idx = rand_batch_idx % len(Input_WHs)
                self.batch_i_to_scale_i[batch_i] = rand_scale_idx
        else:
            self.num_batches = self.nF // self.batch_size + 1
            for batch_i in range(self.num_batches):
                rand_batch_idx = np.random.randint(0, self.num_batches)
                rand_scale_idx = rand_batch_idx % len(self.input_multi_scales)
                self.batch_i_to_scale_i[batch_i] = rand_scale_idx
    
    def gen_multi_scale_input_whs(self, num_scales=256, min_ratio=0.67, max_ratio=1.1):
        """
        generate input multi scale image sizes(w, h), keep default aspect ratio
        :param num_scales:
        :return:
        """
        gs = 32  # grid size

        self.input_multi_scales = [x for x in Input_WHs if not (x[0] % gs or x[1] % gs)]
        self.input_multi_scales.append([self.width, self.height])

        # ----- min scale and max scale
        # keep default aspect ratio
        self.default_aspect_ratio = self.height / self.width

        # min scale
        min_width = math.ceil(self.width * min_ratio / gs) * gs
        min_height = math.ceil(self.height * min_ratio / gs) * gs
        self.input_multi_scales.append([min_width, min_height])

        # max scale
        max_width = math.ceil(self.width * max_ratio / gs) * gs
        max_height = math.ceil(self.height * max_ratio / gs) * gs
        self.input_multi_scales.append([max_width, max_height])

        # other scales
        # widths = list(range(min_width, max_width + 1, int((max_width - min_width) / num_scales)))
        # heights = list(range(min_height, max_height + 1, int((max_height - min_height) / num_scales)))
        widths = list(range(min_width, max_width + 1, 1))
        heights = list(range(min_height, max_height + 1, 1))
        widths = [width for width in widths if not (width % gs)]
        heights = [height for height in heights if not (height % gs)]
        if len(widths) < len(heights):
            for width in widths:
                height = math.ceil(width * self.default_aspect_ratio / gs) * gs
                if [width, height] in self.input_multi_scales:
                    continue
                self.input_multi_scales.append([width, height])
        elif len(widths) > len(heights):
            for height in heights:
                width = math.ceil(height / self.default_aspect_ratio / gs) * gs
                if [width, height] in self.input_multi_scales:
                    continue
                self.input_multi_scales.append([width, height])
        else:
            for width, height in zip(widths, heights):
                if [width, height] in self.input_multi_scales:
                    continue
                height = math.ceil(width * self.default_aspect_ratio / gs) * gs
                self.input_multi_scales.append([width, height])

        if len(self.input_multi_scales) < 2:
            self.input_multi_scales = None
            print('[warning]: generate multi-scales failed(keeping aspect ratio)')
        else:
            self.input_multi_scales.sort(key=lambda x: x[0])
        
    def __getitem__(self, idx):
        batch_i = idx // int(self.opt.batch_size)
        scale_idx = self.batch_i_to_scale_i[batch_i]
        if self.input_multi_scales is None:
            width, height = Input_WHs[scale_idx]
        else:
            width, height = self.input_multi_scales[scale_idx]
        
        for i,c in enumerate(self.cds):
            if idx>c:
                ds = list(self.label_files.keys())[i]
                start_index = c
        
        img_path = self.img_files[ds][idx-start_idx]
        label_path = self.label_files[ds][idx-start_idx]
        
        imgs, labels, img_path, (input_h, input_w) = self.get_data(img_path, label_path,
                                                                  width, height)
        if self.opt.id_weight > 0:
            for i, _ in enumerate(labels):
                for labels[i, 1] > -1:
                    cls_id = int(labels[i][0])
                    start_idx = self.tid_start_idx_of_cls_ids[ds][cls_id]
                    labels[i,1] += start_idx
        
        output_h = img.shape[1]//self.opt.down_ratio 
        output_w = img.shape[2]//self.opt.down_ratio
        
        num_objs = labels.shape[0]
        hm = np.zeros((self.num_classes, output_h, output_w), dtype = np.float32)
        wh = np.zeros((self.max_objs, 2), dtype = np.float32)
        reg = np.zeros((self.max_objs, 2), dtype = np.float32)
        ind = np.zeros((self.max_objs,), dtype = np.int64)
        reg_mask = np.zeros((self.max_objs,), dtype = np.uint8)
        
        if self.opt.id_weight > 0:
            ids = np.zeros((self.max_objs,), dtype = np.int64)
            cls_tr_ids = np.zeros((self.num_classes, output_h, output_w), dtype = np.int64)
            cls_id_map = np.full((1, output_h, output_w), -1, dtype = np.int64)
        draw_gaussian = draw_msra_gaussian if self.opt.mse_loss else draw_umich_gaussian
        
        for k in range(num_objs):
            label = labels[k]
            bbox = label[2:]
            cls_id = int(label[0])
            bbox[[0,2]] = bbox[[0,2]] * output_w
            bbox[[1,3]] = bbox[[1,3]] * output_h
            bbox[0] = np.clip(bbox[0], 0, output_w-1)
            bbox[1] = np.clip(bbox[1], 0, output_h-1)
            
            w,h = bbox[2], bbox[3]
            if h>0 and w>0:
                radius = gaussian_radius((math.ceil(h), math.ceil(w)))
                radius = max(0, int(radius))
                radius = self.opt.hm_gauss if self.opt.mse_loss else radius
                
                ct = np.array([bbox[0], bbox[1]], dtype = np.float32)
                ct_int = ct.astype(np.int32)
                
                draw_gaussian(hm[cls_id], ct_int, radius)
                wh[k] = float(w), float(h)
                ind[k] = ct_int[1] * output_w+ct_int[0]
                reg[k] = ct - ct_int
                reg_mask[k] = 1
                
                if self.opt.id_weight > 0:
                    cls_id_map[0][ct_int[1], ct_int[0]] = cls_id
                    cls_tr_ids[cls_id][ct_int[1]][ct_int[0]] = label[1] - 1
                    ids[k] = label[1] - 1
                
        if self.opt.id_weight > 0:
            ret = {
                'input':img,
                'hm':hm,
                'wh':wh,
                'ind':ind,
                'reg_mask':reg_mask,
                'ids':ids,
                'cls_tr_ids':cls_tr_ids,
                'cls_id_map':cls_id_map
            }
        return ret
                
            
        
        
    

In [2]:
def letterbox(img, height, width, color=(127.5, 127.5, 127.5)):
    shape = img.shape[:2]
    ratio = min(float(height)/shape[0], float(width)/shape[1])
    new_shape = (round(shape[1]*ratio), round(shape[0]*ratio))
    dw = (width - new_shape[0])*0.5
    dh = (height- new_shape[1])*0.5
    top, bottom = round(dw-0.1), round(dw+0.1)
    left, right = round(dw-0.1, dw+0.1)
    
    img = cv2.resize(img, new_shape, interpolation = cv2.INTER_AREA)
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT,
                            value = color)
    return img, ratio, dw, dh

In [ ]:
def xyxy2xywh(x):
    #                                   0   1   2   3
    # Convert bounding box format from [x1, y1, x2, y2] to [x, y, w, h]
    y = torch.zeros(x.shape) if x.dtype is torch.float32 else np.zeros(x.shape)
    y[:, 0] = (x[:, 0] + x[:, 2]) / 2  # center x
    y[:, 1] = (x[:, 1] + x[:, 3]) / 2  # center y
    y[:, 2] = x[:, 2] - x[:, 0]  # width
    y[:, 3] = x[:, 3] - x[:, 1]  # height
    return y

In [ ]:
def random_affine(img, targets=None,
                  degrees=(-10, 10),
                  translate=(.1, .1),
                  scale=(.9, 1.1),
                  shear=(-2, 2),
                  borderValue=(127.5, 127.5, 127.5)):

    border = 0  
    height = img.shape[0]
    width = img.shape[1]

    R = np.eye(3)
    a = random.random() * (degrees[1] - degrees[0]) + degrees[0]
    s = random.random() * (scale[1] - scale[0]) + scale[0]
    R[:2] = cv2.getRotationMatrix2D(angle=a, center=(
        img.shape[1] / 2, img.shape[0] / 2), scale=s)

    T = np.eye(3)
    T[0, 2] = (random.random() * 2 - 1) * translate[0] * \
              img.shape[0] + border  # x translation (pixels)
    T[1, 2] = (random.random() * 2 - 1) * translate[1] * \
              img.shape[1] + border  # y translation (pixels)

    #错切
    S = np.eye(3)
    S[0, 1] = math.tan((random.random() * (shear[1] - shear[0]) +
                        shear[0]) * math.pi / 180)  # x shear (deg)
    S[1, 0] = math.tan((random.random() * (shear[1] - shear[0]) +
                        shear[0]) * math.pi / 180)  # y shear (deg)

    M = S @ T @ R  #matmul
    imw = cv2.warpPerspective(img, M, dsize=(width, height), flags=cv2.INTER_LINEAR,
                              borderValue=borderValue)  # BGR order borderValue

    # Return warped points also
    if targets is not None:
        if len(targets) > 0:
            n = targets.shape[0]
            points = targets[:, 2:6].copy()
            area0 = (points[:, 2] - points[:, 0]) * \
                    (points[:, 3] - points[:, 1])

            # warp points
            xy = np.ones((n * 4, 3))
            xy[:, :2] = points[:, [0, 1, 2, 3, 0, 3, 2, 1]].reshape(
                n * 4, 2)  # x1y1, x2y2, x1y2, x2y1
            xy = (xy @ M.T)[:, :2].reshape(n, 8)

            # create new boxes
            x = xy[:, [0, 2, 4, 6]]
            y = xy[:, [1, 3, 5, 7]]
            xy = np.concatenate(
                (x.min(1), y.min(1), x.max(1), y.max(1))).reshape(4, n).T

            # apply angle-based reduction
            radians = a * math.pi / 180
            reduction = max(abs(math.sin(radians)),
                            abs(math.cos(radians))) ** 0.5
            x = (xy[:, 2] + xy[:, 0]) / 2
            y = (xy[:, 3] + xy[:, 1]) / 2
            w = (xy[:, 2] - xy[:, 0]) * reduction
            h = (xy[:, 3] - xy[:, 1]) * reduction
            xy = np.concatenate((x - w / 2, y - h / 2, x + w / 2, y + h / 2)).
            reshape(4, n).T

            # reject warped points outside of image
            np.clip(xy[:, 0], 0, width, out=xy[:, 0])
            np.clip(xy[:, 2], 0, width, out=xy[:, 2])
            np.clip(xy[:, 1], 0, height, out=xy[:, 1])
            np.clip(xy[:, 3], 0, height, out=xy[:, 3])
            w = xy[:, 2] - xy[:, 0]
            h = xy[:, 3] - xy[:, 1]
            area = w * h
            ar = np.maximum(w / (h + 1e-16), h / (w + 1e-16))
            i = (w > 4) & (h > 4) & (area / (area0 + 1e-16) > 0.1) & (ar < 10)

            targets = targets[i]
            targets[:, 2:6] = xy[i]

        return imw, targets, M
    else:
        return imw